# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [1]:
llm_config={"model": "gpt-3.5-turbo"}

In [2]:
from autogen import ConversableAgent

In [9]:
def is_terminator(msg):
    done = False
    for word in ["terminate", "bye", "got to go"]:
        done = word in msg.get("content").lower()
        if done: 
            return done
    return done

## Creating the needed agents

In [4]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [10]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: is_terminator(msg),
)

In [11]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: is_terminator(msg),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [12]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': '', 'topic': ''}",
        },
        "max_turns": 2,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 2,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': '', 'topic': '','fun_fact': '',}",
        },
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [14]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/usr/local/lib/python3.11/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: bye
customer_proxy_agent (to Onboarding Personal Information Agent):

bye

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

I'm sorry to see you go. If you change your mind and need assistance, feel free to reach out. Goodbye!

--------------------------------------------------------------------------------
Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what

## Print out the summary

In [15]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': '', 'location': ''}


{'name': 'wen', 'location': 'cary', 'topic': 'apple vision'}


{'name': '', 'location': '', 'topic': 'smartphone','fun_fact': 'The first smartphone ever created was the IBM Simon Personal Communicator in 1994.'}




## Print out the cost

In [16]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.000289, 'gpt-3.5-turbo-0125': {'cost': 0.000289, 'prompt_tokens': 374, 'completion_tokens': 68, 'total_tokens': 442}}, 'usage_excluding_cached_inference': {'total_cost': 0.000289, 'gpt-3.5-turbo-0125': {'cost': 0.000289, 'prompt_tokens': 374, 'completion_tokens': 68, 'total_tokens': 442}}}


{'usage_including_cached_inference': {'total_cost': 0.00036, 'gpt-3.5-turbo-0125': {'cost': 0.00036, 'prompt_tokens': 528, 'completion_tokens': 64, 'total_tokens': 592}}, 'usage_excluding_cached_inference': {'total_cost': 0.00036, 'gpt-3.5-turbo-0125': {'cost': 0.00036, 'prompt_tokens': 528, 'completion_tokens': 64, 'total_tokens': 592}}}


{'usage_including_cached_inference': {'total_cost': 0.0006725, 'gpt-3.5-turbo-0125': {'cost': 0.0006725, 'prompt_tokens': 643, 'completion_tokens': 234, 'total_tokens': 877}}, 'usage_excluding_cached_inference': {'total_cost': 0.0006725, 'gpt-3.5-turbo-0125': {'cost': 0.0006725, 'prompt_tokens': 643, 'complet